*Start up connection to VND session with...*

In [ ]:
import VND


*Now make a selction object using the VND selection langugage...*

In [ ]:
mySel=VND.NeuronSel('(soma x < -15) && (soma y < 0) && (population == internal)')



*Check the status of this object, and see how many neurons are in the selection*

In [ ]:
mySel

*Make a new graphical representation, which will appear in the VND 3D display and be listed in the VND main tab Representations list:*


In [ ]:
VND.add_rep(sel=mySel, color='red', style='soma', scaling='1.8', resolution="10", material='Opaque', show='True')


*List some attribute values of your selection...*

In [ ]:
mySel.get(['x','node', 'model_name'])


In [ ]:
#Show some autocompletion
VND.list_attributes()


*Use  simple .get queries, connected to selections and GUI representations, to populate Pandas DataFrames...*

In [ ]:
internalSel=VND.NeuronSel('population == internal ')


In [ ]:
internalSel

In [ ]:
attribs=VND.list_attributes()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#choose a random 10 perent of the excitatory cells
exc_cell_ids = internal_nodes_df[internal_nodes_df['ei'] == 'e']['node_id'].values
exc_cell_ids = np.random.choice(exc_cell_ids, size=int(len(exc_cell_ids)*.1), replace=False)


In [ ]:
#combine column data and titles for the columns
internal_nodes_df=pd.DataFrame(internalSel.get(attribs),columns=attribs)


In [ ]:
#choose a random 10 perent of the excitatory cells
exc_cell_ids = internal_nodes_df[internal_nodes_df['ei'] == 'e']['node_id'].values
exc_cell_ids = np.random.choice(exc_cell_ids, size=int(len(exc_cell_ids)*.1), replace=False)


In [ ]:
exc_cell_ids

In [ ]:
inh_cell_ids = internal_nodes_df[internal_nodes_df['ei'] == 'i']['node_id'].values
inh_cell_ids = np.random.choice(inh_cell_ids, size=int(len(inh_cell_ids)*.1), replace=False)


In [ ]:
inhSel=VND.NeuronSel(f'(node_id == {" ".join(inh_cell_ids)}) && (population == internal)')

In [ ]:
inhSel

In [ ]:
excSel=VND.NeuronSel(f'(node_id == {" ".join(exc_cell_ids)}) && (population == internal)')

In [ ]:
excSel

In [ ]:
VND.add_rep(sel=excSel, color='pink', style='soma', scaling='2.0', material='Opaque', resolution='15',show='True')


In [ ]:
VND.add_rep(sel=inhSel, color='iceblue', style='morphology', scaling='1.6', material='Opaque', show='True')


*Work with spike (action potential) events...*

In [ ]:
internSel=VND.NeuronSel('population == internal')
d = internSel.get_events_dict("internal")
spikes_df = pd.DataFrame({
    'node_id':d['n'],
    'timestamps':d['t']
})
spikes_df

*Find cells with the highest spiking rates:*

In [ ]:
# group df by id and use 'count' function to determine firing rates, 
# sort dataframe by firing rate and get the top 10
min_ts, max_ts = 0, np.ceil(spikes_df['timestamps'].max())
firing_rates_df = (spikes_df.groupby('node_id').agg('count')/(max_ts - min_ts)*1000.0).reset_index()
firing_rates_df = firing_rates_df.rename(columns={'timestamps': 'firing_rate'})
firing_rates_df = firing_rates_df.sort_values('firing_rate', ascending=False)
fastest_cell_ids = firing_rates_df['node_id'].head(10).values
fastest_cell_ids_seltext=' '.join(map(str, fastest_cell_ids))

In [ ]:
fastest_cell_ids_seltext

In [ ]:
fastestSel=VND.NeuronSel(f'(node_id = {fastest_cell_ids_seltext} ) && (population == internal)')

In [ ]:
fastestSel

In [ ]:
fastestSel.get(['node', 'ei', 'model_name'])


*Animated spike events in activity window...*

In [ ]:
#Make a rep for the  the fastestSel selection; then animate it in the Activity window
VND.add_rep(sel=fastestSel, color='red', style='soma', scaling='1.8', resolution=6, material='Opaque', show='True')

*Use selection objects to draw interactive raster spike plot, examine highlighted events:*

In [ ]:
from bokeh.models import ColumnDataSource, NumeralTickFormatter, LassoSelectTool, HoverTool, TapTool
from bokeh.plotting import figure, show
from bokeh.events import Tap, SelectionGeometry
from bokeh.models.callbacks import CustomJS
from bokeh.io import reset_output,output_notebook

plottingSel = VND.NeuronSel('population == internal')
#Make a rep for the  the plottingSel selection; then animate it in the Activity window


dynSel = VND.NeuronSel('! (all)')
dyn_plot_rep=VND.add_rep(sel=dynSel, color='red', style='soma', scaling='1.2', resolution="6", material='Transparent', show='True')
print (f'dyn_plot_rep is {dyn_plot_rep}')
vndSpikeDataTransfer = plottingSel.get_events_plot_dict("internal")
#now clear 
reset_output()
output_notebook()

source = ColumnDataSource(vndSpikeDataTransfer)
selected = ColumnDataSource(dict(
    x=[],
    y=[],
))

fig = figure(title='VND Spike Graph: Select the events',
             plot_height=500, plot_width=600,
             x_range=(0, max(vndSpikeDataTransfer["x"])), y_range=(0, max(vndSpikeDataTransfer["y"])))

hover_glyph = fig.circle(source=source, x='x', y='y',
           color='c', size=3, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('t (ms)', '@x'), ('node', '@y'), ('c', '@c'), ('global_id', '@g')]

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
fig.add_tools(TapTool())
fig.add_tools(LassoSelectTool())

In [ ]:
callback_4 = CustomJS(args=dict(source=source,dyn_plot_rep=dyn_plot_rep), code="""
    // get data source from Callback args
    let data = Object.assign({}, source.data);
    var kernel = Jupyter.notebook.kernel;    
    var inds = source.selected.indices;
    let indices = Object.assign([], source.selected.indices);
    data.g =indices.map(i=>data.g[i]);
    var data_g_str = data.g.toString();
    var data_g_repl = data_g_str.replace(/,/g," ");
    var command = "VND.vndSocketReq(f'::neuro::cmd_mod_rep_node_gid_list {dyn_plot_rep} soma 1 Transparent 1.2 6 [list "+ data_g_repl + "]')";
    var command2 = `dyn_plot_highlight="${data_g_repl}"`  
    kernel.execute(command)
    kernel.execute(command2)
""") 

fig.js_on_event(SelectionGeometry,  callback_4)

           
show(fig)

*Make a selection from the dynamic plot selection...*

In [ ]:
dynSel = VND.NeuronSel(f'gid == {dyn_plot_highlight}')
dynSel


In [ ]:
dynSel.get(['node_id', 'ei', 'model_name'])

In [ ]:
attribs=VND.list_attributes()
dyn_nodes_df=pd.DataFrame(dynSel.get(attribs),columns=attribs)
dyn_nodes_df

In [ ]:
dyn_nodes_df['model_name'].value_counts()

*Make a lot for the fastest firing neurons only, using the ``fastestSel`` selection...*

In [ ]:
from bokeh.models import ColumnDataSource, NumeralTickFormatter, LassoSelectTool, HoverTool, TapTool
from bokeh.plotting import figure, show
from bokeh.events import Tap, SelectionGeometry
from bokeh.models.callbacks import CustomJS
from bokeh.io import reset_output,output_notebook

#fastestSel = VND.NeuronSel('population == internal')
#Make a rep for the  the plottingSel selection; then animate it in the Activity window


dynSel = VND.NeuronSel('! (all)')
dyn_plot_rep=VND.add_rep(sel=dynSel, color='red', style='soma', scaling='1.2', resolution="6", material='Transparent', show='True')
print (f'dyn_plot_rep is {dyn_plot_rep}')
#here we use the fastestSel selection object...
vndSpikeDataTransfer2 = fastestSel.get_events_plot_dict("internal")
#now clear 
reset_output()
output_notebook()

source = ColumnDataSource(vndSpikeDataTransfer2)
selected = ColumnDataSource(dict(
    x=[],
    y=[],
))

fig = figure(title='VND Spike Graph: Select the events',
             plot_height=500, plot_width=600,
             x_range=(0, max(vndSpikeDataTransfer2["x"])), y_range=(0, max(vndSpikeDataTransfer["y"])))

hover_glyph = fig.circle(source=source, x='x', y='y',
           color='c', size=3, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('t (ms)', '@x'), ('node', '@y'), ('c', '@c'), ('global_id', '@g')]

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
fig.add_tools(TapTool())
fig.add_tools(LassoSelectTool())

In [ ]:
callback_4 = CustomJS(args=dict(source=source,dyn_plot_rep=dyn_plot_rep), code="""
    // get data source from Callback args
    let data = Object.assign({}, source.data);
    var kernel = Jupyter.notebook.kernel;    
    var inds = source.selected.indices;
    let indices = Object.assign([], source.selected.indices);
    data.g =indices.map(i=>data.g[i]);
    var data_g_str = data.g.toString();
    var data_g_repl = data_g_str.replace(/,/g," ");
    var command = "VND.vndSocketReq(f'::neuro::cmd_mod_rep_node_gid_list {dyn_plot_rep} soma 1 Transparent 1.2 6 [list "+ data_g_repl + "]')";
    var command2 = `dyn_plot_highlight="${data_g_repl}"`  
    kernel.execute(command)
    kernel.execute(command2)
""") 

fig.js_on_event(SelectionGeometry,  callback_4)

           
show(fig)

In [ ]:
dd = mySel.get_events_dict("internal")

In [ ]:
dd

In [ ]:
gdd=pd.DataFrame(dd)

In [ ]:
gdd

In [ ]:
VND.add_rep(sel=fastestSel)

In [ ]:
newSel=NeuronSel('')